Artificial Neural Network

In [8]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt

def load_data(filepath):
    r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
    return pd.read_csv(filepath, sep='\t', names=r_cols, encoding='latin-1')

def build_model(num_users, num_movies, embedding_size=50):
    user_input = Input(shape=(1,), name='user_input')
    user_embedding = Embedding(input_dim=num_users+1, output_dim=embedding_size, name='user_embedding')(user_input)
    user_vec = Flatten(name='flatten_user')(user_embedding)

    movie_input = Input(shape=(1,), name='movie_input')
    movie_embedding = Embedding(input_dim=num_movies+1, output_dim=embedding_size, name='movie_embedding')(movie_input)
    movie_vec = Flatten(name='flatten_movie')(movie_embedding)

    concat = Concatenate()([user_vec, movie_vec])
    dense = Dense(256, activation='relu')(concat)
    batch_norm = BatchNormalization()(dense)
    dropout = Dropout(0.5)(batch_norm)
    output = Dense(1)(dropout)

    model = Model(inputs=[user_input, movie_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')
    return model

def main():
    # Determine the maximum number of users and movies to set the embedding input dimensions
    ratings = pd.concat([load_data(f'ml-100k/u{k}.base') for k in range(1, 6)])
    num_users = ratings['user_id'].max()
    num_movies = ratings['movie_id'].max()

    model = build_model(num_users, num_movies)

    scaler = StandardScaler()

    rmses = []
    accuracies = []

    # Loop over each set of predefined splits
    for k in range(1, 6):
        train_data = load_data(f'ml-100k/u{k}.base')
        test_data = load_data(f'ml-100k/u{k}.test')

        # Preprocess ratings
        y_train = scaler.fit_transform(train_data['rating'].values.reshape(-1, 1))
        y_test = test_data['rating'].values

        # Train model
        model.fit([train_data['user_id'], train_data['movie_id']], y_train, 
                  batch_size=32, epochs=5, verbose=0)

        # Predict and evaluate
        preds = scaler.inverse_transform(model.predict([test_data['user_id'], test_data['movie_id']]))
        rmse = sqrt(mean_squared_error(y_test, preds))
        accuracy = np.mean(np.abs(preds.ravel() - y_test) <= 0.7)

        rmses.append(rmse)
        accuracies.append(accuracy)

    print(f"Mean RMSE across 5 folds: {np.mean(rmses):.3f} ± {np.std(rmses):.3f}")
    print(f"Accuracy (within 0.5 of true rating): {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")

if __name__ == "__main__":
    main()


625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 887us/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 862us/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 830us/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 957us/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 949us/step
Mean RMSE across 5 folds: 0.836 ± 0.073
Accuracy (within 0.5 of true rating): 0.615 ± 0.044


In [ ]:
Gradient Boosting Regressor

In [22]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

def load_data(base_path, test_path):
    # Load base and test data
    base_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
    test_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

    base_data = pd.read_csv(base_path, sep='\t', names=base_cols)
    test_data = pd.read_csv(test_path, sep='\t', names=test_cols)

    return base_data, test_data

def prepare_features(data):
    # Assuming that we have user and movie average ratings calculated
    data['user_avg'] = data.groupby('user_id')['rating'].transform('mean')
    data['movie_avg'] = data.groupby('movie_id')['rating'].transform('mean')
    data['interaction'] = data['user_avg'] * data['movie_avg']

    return data[['user_avg', 'movie_avg', 'interaction', 'rating']]

def train_and_evaluate(train_data, test_data):
    # Prepare features
    X_train = train_data[['user_avg', 'movie_avg', 'interaction']]
    y_train = train_data['rating']
    X_test = test_data[['user_avg', 'movie_avg', 'interaction']]
    y_test = test_data['rating']

    # Train Gradient Boosting Regressor
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    # Calculate RMSE and accuracy
    rmse = sqrt(mean_squared_error(y_test, preds))
    accuracy = np.mean(np.abs(preds - y_test) <= 0.5)

    return rmse, accuracy

def main():
    rmses = []
    accuracies = []

    # Iterate over each of the 5 predefined splits
    for k in range(1, 6):
        base_path = f'ml-100k/u{k}.base'
        test_path = f'ml-100k/u{k}.test'
        train_data, test_data = load_data(base_path, test_path)
        
        # Feature engineering
        train_data = prepare_features(train_data)
        test_data = prepare_features(test_data)
        
        rmse, accuracy = train_and_evaluate(train_data, test_data)
        rmses.append(rmse)
        accuracies.append(accuracy)
    
    # Output mean RMSE and Accuracy across 5 folds
    print(f"Mean RMSE across 5 folds: {np.mean(rmses):.3f} ± {np.std(rmses):.3f}")
    print(f"Accuracy (within 0.5 of true rating): {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")

if __name__ == "__main__":
    main()


Mean RMSE across 5 folds: 0.895 ± 0.010
Accuracy (within 0.5 of true rating): 0.433 ± 0.004


In [ ]:
Ridge

In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt

def load_data(filepath):
    r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
    return pd.read_csv(filepath, sep='\t', names=r_cols, encoding='latin-1')

def load_item_data():
    i_cols = ['movie_id', 'movie_title', 'release_date', 'video_release_date',
              'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's",
              'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
    items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1', usecols=range(24))
    tfidf_transformer = TfidfTransformer(smooth_idf=True)
    genre_matrix = items.iloc[:, 5:].fillna(0)
    tfidf_features = tfidf_transformer.fit_transform(genre_matrix).toarray()
    return items, tfidf_features

def main():
    items, items_tfidf = load_item_data()
    scaler = StandardScaler()

    rmses = []
    accuracies = []

    # 5-fold cross-validation process
    for k in range(1, 6):  # For each of the 5 folds
        train_path = f'ml-100k/u{k}.base'  # Correct file name for training data
        test_path = f'ml-100k/u{k}.test'  # Correct file name for test data
        train_data = load_data(train_path)
        test_data = load_data(test_path)

        # Merge TF-IDF features from the item data
        train_data = train_data.join(items.set_index('movie_id'), on='movie_id')
        test_data = test_data.join(items.set_index('movie_id'), on='movie_id')

        # Prepare features and targets
        X_train = items_tfidf[train_data['movie_id'] - 1]
        X_test = items_tfidf[test_data['movie_id'] - 1]
        y_train = scaler.fit_transform(train_data[['rating']])
        y_test = test_data['rating']

        # Model training
        model = Ridge(alpha=0.01)
        model.fit(X_train, y_train.ravel())

        # Predict and evaluate
        preds = scaler.inverse_transform(model.predict(X_test).reshape(-1, 1))
        rmse = sqrt(mean_squared_error(y_test, preds))
        rmses.append(rmse)
        accuracy = np.mean(np.abs(preds.ravel() - y_test) <= 0.5)
        accuracies.append(accuracy)

    # Output results
    print(f"Mean RMSE across 5 folds: {np.mean(rmses):.3f} ± {np.std(rmses):.3f}")
    print(f"Accuracy (within 0.5 of true rating): {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")

if __name__ == "__main__":
    main()

Mean RMSE across 5 folds: 1.107 ± 0.015
Accuracy (within 0.5 of true rating): 0.334 ± 0.004


In [39]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from math import sqrt

def load_and_preprocess_data():
    # Load Ratings
    ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')
    
    # Load User Info
    users = pd.read_csv('ml-100k/u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'], engine='python')
    
    # Load Movie Info; ensure headers are correctly processed
    movies = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', names=['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], usecols=range(24), engine='python')
    
    # Merge data
    data = ratings.merge(users, on='user_id').merge(movies, on='movie_id')
    
    # Handling categorical data: One-hot encoding for 'gender' and 'occupation'
    categorical_cols = ['gender', 'occupation']
    data = pd.get_dummies(data, columns=categorical_cols)
    
    # Normalize numeric columns: 'age' and perhaps other continuous variables
    scaler = StandardScaler()
    numeric_cols = ['age']  # Add other numeric columns if needed
    data[numeric_cols] = scaler.fit_transform(data[numeric_cols])
    
    return data

# def main():
#     data = load_and_preprocess_data()
#     print(data.head())

# if __name__ == "__main__":
#     main()


In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from math import sqrt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout, Concatenate, Multiply, BatchNormalization
from tensorflow.keras.optimizers import Adam

def load_data(fold):
    # Load training and testing data
    train_data = pd.read_csv(f'ml-100k/u{fold}.base', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')
    test_data = pd.read_csv(f'ml-100k/u{fold}.test', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')
    
    # Load user and item data
    users = pd.read_csv('ml-100k/u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'], engine='python')
    items = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], usecols=range(24), engine='python')
    
    # Merge data
    train_data = train_data.merge(users, on='user_id').merge(items, on='movie_id')
    test_data = test_data.merge(users, on='user_id').merge(items, on='movie_id')
    return train_data, test_data

def preprocess_data(train_data, test_data):
    # Ensure indices are reset to prevent InvalidIndexError during concatenation
    train_data.reset_index(drop=True, inplace=True)
    test_data.reset_index(drop=True, inplace=True)

    # Concatenate train and test data
    combined = pd.concat([train_data, test_data], ignore_index=True)

    # Encode categorical variables
    encoder = OneHotEncoder(sparse_output=False)  # Outputs dense array directly
    categorical_features = ['gender', 'occupation']  # Ensure these columns exist
    categorical_encoded = encoder.fit_transform(combined[categorical_features])  # No need for .toarray()

    # Create DataFrame for the encoded features
    categorical_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(), index=combined.index)

    # Concatenate encoded features back to the combined DataFrame
    combined = pd.concat([combined.drop(categorical_features, axis=1), categorical_df], axis=1)

    # Scale numeric features
    scaler = StandardScaler()
    numeric_features = ['age']  # Ensure this column exists
    combined[numeric_features] = scaler.fit_transform(combined[numeric_features])

    # Separate back into train and test datasets based on the length of the original train data
    train_processed = combined.iloc[:len(train_data)]
    test_processed = combined.iloc[len(train_data):]
    return train_processed, test_processed


def build_advanced_model(num_users, num_movies, num_features):
    user_input = Input(shape=(1,), dtype='int32')
    movie_input = Input(shape=(1,), dtype='int32')
    user_embedding = Embedding(num_users + 1, 30, embeddings_regularizer=l2(1e-5))(user_input)
    movie_embedding = Embedding(num_movies + 1, 30, embeddings_regularizer=l2(1e-5))(movie_input)
    
    user_vec = Flatten()(user_embedding)
    movie_vec = Flatten()(movie_embedding)
    interact = Multiply()([user_vec, movie_vec])

    additional_features_input = Input(shape=(num_features,), dtype='float32')
    concat = Concatenate()([interact, additional_features_input])

    dense1 = Dense(128, activation='relu')(concat)
    batch_norm1 = BatchNormalization()(dense1)
    dropout1 = Dropout(0.5)(batch_norm1)
    dense2 = Dense(64, activation='relu')(dropout1)
    batch_norm2 = BatchNormalization()(dense2)
    dropout2 = Dropout(0.3)(batch_norm2)
    output = Dense(1, activation='linear')(dropout2)

    model = Model(inputs=[user_input, movie_input, additional_features_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    return model

def train_and_evaluate_model():
    rmses = []
    accuracies = []

    for fold in range(1, 6):
        train_data, test_data = load_data(fold)
        train_processed, test_processed = preprocess_data(train_data, test_data)
        num_users = max(train_processed['user_id'].max(), test_processed['user_id'].max())
        num_movies = max(train_processed['movie_id'].max(), test_processed['movie_id'].max())

        model = build_model(num_users, num_movies, train_processed.drop(['user_id', 'movie_id', 'rating', 'timestamp'], axis=1).shape[1])

        model.fit([train_processed['user_id'], train_processed['movie_id']], train_processed['rating'], epochs=10, batch_size=32, verbose=0)

        preds = model.predict([test_processed['user_id'], test_processed['movie_id']])
        rmse = sqrt(mean_squared_error(test_processed['rating'], preds))
        accuracy = np.mean(np.abs(preds.flatten() - test_processed['rating']) <= 0.5)

        rmses.append(rmse)
        accuracies.append(accuracy)

    print(f"Final RMSE across 5 folds: {np.mean(rmses):.3f} ± {np.std(rmses):.3f}")
    print(f"Accuracy (within ±0.5 of true rating): {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")

if __name__ == "__main__":
    train_and_evaluate_model()


625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 814us/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 819us/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 811us/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 818us/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 811us/step
Final RMSE across 5 folds: 0.927 ± 0.006
Accuracy (within ±0.5 of true rating): 0.425 ± 0.004


In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout, Concatenate, Multiply, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from math import sqrt
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout, Flatten, Concatenate, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def load_data(fold):
    # Load training and testing data
    train_data = pd.read_csv(f'ml-100k/u{fold}.base', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')
    test_data = pd.read_csv(f'ml-100k/u{fold}.test', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')
    
    # Load user and item data
    users = pd.read_csv('ml-100k/u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'], engine='python')
    items = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], usecols=range(24), engine='python')
    
    # Merge data
    train_data = train_data.merge(users, on='user_id').merge(items, on='movie_id')
    test_data = test_data.merge(users, on='user_id').merge(items, on='movie_id')
    return train_data, test_data

def preprocess_data(train_data, test_data, exclude_scaling):
    train_data.reset_index(drop=True, inplace=True)
    test_data.reset_index(drop=True, inplace=True)

    combined = pd.concat([train_data, test_data], ignore_index=True)

    # One-hot encode categorical variables
    categorical_features = ['gender', 'occupation']  # Adjust as needed
    encoder = OneHotEncoder(sparse_output=False)
    categorical_encoded = encoder.fit_transform(combined[categorical_features])
    categorical_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(), index=combined.index)

    combined.drop(categorical_features, axis=1, inplace=True)
    combined = pd.concat([combined, categorical_df], axis=1)

    numeric_features = [col for col in combined.columns if combined[col].dtype in [np.float64, np.int64] and col not in exclude_scaling]
    scaler = StandardScaler()
    combined[numeric_features] = scaler.fit_transform(combined[numeric_features])

    train_processed = combined.iloc[:len(train_data)]
    test_processed = combined.iloc[len(train_data):]
    return train_processed, test_processed

def build_more_complex_model(num_users, num_movies, num_features):
    # User and Movie embeddings
    user_input = Input(shape=(1,), dtype='int32')
    movie_input = Input(shape=(1,), dtype='int32')
    user_embedding = Embedding(num_users + 1, 50, name="user_embedding")(user_input)
    movie_embedding = Embedding(num_movies + 1, 50, name="movie_embedding")(movie_input)

    # Flattening embedded layers
    user_vec = Flatten()(user_embedding)
    movie_vec = Flatten()(movie_embedding)

    # Combine features with additional input features
    additional_features_input = Input(shape=(num_features,), dtype='float32')
    concat = Concatenate()([user_vec, movie_vec, additional_features_input])

    # Neural network layers
    dense = Dense(256, activation='relu')(concat)
    batch_norm = BatchNormalization()(dense)
    dropout = Dropout(0.5)(batch_norm)
    final_layer = Dense(128, activation='relu')(dropout)
    output = Dense(1, activation='linear')(final_layer)

    model = Model(inputs=[user_input, movie_input, additional_features_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

    return model

def train_and_evaluate_model():
    rmses = []
    accuracies = []

    # List of columns to exclude from feature input and scaling
    exclude_scaling = ['movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'zip_code']

    for fold in range(1, 6):
        train_data, test_data = load_data(fold)
        train_processed, test_processed = preprocess_data(train_data, test_data, exclude_scaling)
        
        num_users = train_processed['user_id'].nunique()
        num_movies = train_processed['movie_id'].nunique()
        feature_cols = [col for col in train_processed.columns if col not in ['user_id', 'movie_id', 'rating', 'timestamp'] + exclude_scaling]

        model = build_advanced_model(num_users, num_movies, len(feature_cols))
        model.fit([train_processed['user_id'], train_processed['movie_id'], train_processed[feature_cols]], train_processed['rating'], epochs=10, batch_size=32, verbose=1)

        preds = model.predict([test_processed['user_id'], test_processed['movie_id'], test_processed[feature_cols]])
        rmse = sqrt(mean_squared_error(test_processed['rating'], preds))
        accuracy = np.mean(np.abs(preds.flatten() - test_processed['rating']) <= 0.5)

        rmses.append(rmse)
        accuracies.append(accuracy)

    print(f"Final RMSE across 5 folds: {np.mean(rmses):.3f} ± {np.std(rmses):.3f}")
    print(f"Accuracy (within ±0.5 of true rating): {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")

if __name__ == "__main__":
    train_and_evaluate_model()

Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.0201
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 965us/step - loss: 0.9156
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.8974
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.8895
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.8849
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.8841
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.8722
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.8821
Epoch 9/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.8728
Epoch 10/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.8682
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.0309
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.9266
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.9183
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - los